In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import axis as ax
from gsw import SA_from_SP, CT_from_t, rho, p_from_z, spiciness0
%matplotlib inline

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [2]:
cd "/Users/alaimo/Documents/Rutgers/RoadToPhD/Chapter 2"

/Users/alaimo/Documents/Rutgers/RoadToPhD/Chapter 2


In [3]:
neamap = pd.read_csv("Data/neamap_full.csv")

In [4]:
neamap

,Unnamed: 0,cruise,station,year,season,date,time,region,dstrat,lat,...,areasw,sWT,sSA,sDO,bWT,bSA,bDO,vimscode,count,weight
0,151,NM080401,NM20080401001,2008,SPRING,2008-05-15,05:49:00,01,2,41.012000,...,0.024644,10.19,29.13,8.84,9.60,29.59,8.59,1028,468,4.845
1,152,NM080401,NM20080401002,2008,SPRING,2008-05-08,09:20:00,01,2,41.022333,...,0.020634,10.14,28.59,9.77,9.20,29.38,9.63,1028,379,3.580
2,153,NM080401,NM20080401003,2008,SPRING,2008-05-15,07:19:00,02,1,40.981500,...,0.021517,9.88,29.46,8.11,9.66,29.52,8.00,1028,216,0.904
3,154,NM080401,NM20080401004,2008,SPRING,2008-05-15,12:17:00,02,1,40.835000,...,0.026516,11.16,28.69,7.79,9.31,30.02,6.97,1028,189,1.645
4,155,NM080401,NM20080401005,2008,SPRING,2008-05-15,08:49:00,02,2,40.909333,...,0.026212,11.13,28.81,8.01,9.54,29.91,7.50,1028,98,1.992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2317,4878,NM240401,NM20240401146,2024,SPRING,2024-05-24,15:09:00,RI,4,41.414200,...,0.025642,14.19,29.83,7.45,10.66,30.54,8.18,1028,26,1.350
2318,4879,NM240401,NM20240401147,2024,SPRING,2024-05-26,17:17:12,RI,4,41.327800,...,0.026303,14.67,29.62,4.28,9.57,30.53,7.31,1028,81,1.295
2319,4880,NM240401,NM20240401148,2024,SPRING,2024-05-26,15:19:52,RI,4,41.150800,...,0.025763,14.99,30.03,4.01,7.40,30.91,7.44,1028,64,2.440
2320,4881,NM240401,NM20240401149,2024,SPRING,2024-05-26,12:50:56,RI,4,41.262200,...,0.025156,14.56,29.47,3.88,9.37,30.63,7.74,1028,205,3.731


In [5]:
neamap = neamap.rename(columns = {"weight" :"squid_weight", "count":"squid",'lat':"SLAT","lon":"SLONG", "bWT":"TEMPBOT","bSA":"SALBOT","bDO":"DOBOT", "sWT":"TEMPSURF","sSA":"SALSURF","sDO":"DOSURF"})
neamap['squid1'] = np.where(neamap['squid'] >= 1.0,1,0).astype(int)

In [6]:
neamap['date'] = pd.to_datetime(neamap['date'], format='%Y-%m-%d')

In [7]:
neamap["DOY"] = pd.to_datetime(neamap['date'], format='%Y-%m-%d').dt.dayofyear

In [9]:
def density(temperature, depth, salinity, latitude, longitude):
    """
    Calculates density given practical salinity, depth, latitude,
    and longitude using Gibbs gsw SA_from_SP and rho functions.
    Args:
        temperature (_type_): temperature (C)
        depth (_type_): depth, positive up (m)
        salinity (array): salinity
        latitude (array): latitude (decimal degrees)
        longitude (array): longitude (decimal degrees)
    Returns:
        density: Density calculated using the Gibbs GSW
    """
    # Calculates sea pressure from height using computationally-efficient
    # 75-term expression for density, in terms of SA, CT and p
    # (Roquet et al., 2015).
    pressure = p_from_z(
        depth,
        latitude,
    )
    # Calculates Absolute Salinity from Practical Salinity.
    # Since SP is non-negative by definition,
    # this function changes any negative input values of SP to be zero.
    absolute_salinity = SA_from_SP(
        salinity,
        pressure,
        longitude,
        latitude
    )
    # Calculates Conservative Temperature of seawater from in-situ temperature.
    conservative_temperature = CT_from_t(
        absolute_salinity,
        temperature,
        pressure
    )
    # Calculates in-situ density from Absolute Salinity and
    # Conservative Temperature, using the computationally-efficient expression
    # for specific volume in terms of SA, CT and p (Roquet et al., 2015).
    density = rho(
        absolute_salinity,
        conservative_temperature,
        pressure
    )
    return density

In [10]:
neamap['denssurf'] = density(neamap.TEMPSURF,-neamap.depth, neamap.SALSURF,neamap.SLAT,neamap.SLONG)

In [11]:
neamap['densbot'] = density(neamap.TEMPBOT,-neamap.depth, neamap.SALBOT,neamap.SLAT,neamap.SLONG)

In [12]:
neamap

,Unnamed: 0,cruise,station,year,season,date,time,region,dstrat,SLAT,...,TEMPBOT,SALBOT,DOBOT,vimscode,squid,squid_weight,squid1,DOY,denssurf,densbot
0,151,NM080401,NM20080401001,2008,SPRING,2008-05-15,05:49:00,01,2,41.012000,...,9.60,29.59,8.59,1028,468,4.845,1,136,1022.411448,1022.863029
1,152,NM080401,NM20080401002,2008,SPRING,2008-05-08,09:20:00,01,2,41.022333,...,9.20,29.38,9.63,1028,379,3.580,1,129,1021.997476,1022.758732
2,153,NM080401,NM20080401003,2008,SPRING,2008-05-15,07:19:00,02,1,40.981500,...,9.66,29.52,8.00,1028,216,0.904,1,136,1022.702682,1022.783906
3,154,NM080401,NM20080401004,2008,SPRING,2008-05-15,12:17:00,02,1,40.835000,...,9.31,30.02,6.97,1028,189,1.645,1,136,1021.903816,1023.237811
4,155,NM080401,NM20080401005,2008,SPRING,2008-05-15,08:49:00,02,2,40.909333,...,9.54,29.91,7.50,1028,98,1.992,1,136,1022.024539,1023.139085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2317,4878,NM240401,NM20240401146,2024,SPRING,2024-05-24,15:09:00,RI,4,41.414200,...,10.66,30.54,8.18,1028,26,1.350,1,145,1022.307622,1023.513470
2318,4879,NM240401,NM20240401147,2024,SPRING,2024-05-26,17:17:12,RI,4,41.327800,...,9.57,30.53,7.31,1028,81,1.295,1,147,1022.059757,1023.695694
2319,4880,NM240401,NM20240401148,2024,SPRING,2024-05-26,15:19:52,RI,4,41.150800,...,7.40,30.91,7.44,1028,64,2.440,1,147,1022.338966,1024.341396
2320,4881,NM240401,NM20240401149,2024,SPRING,2024-05-26,12:50:56,RI,4,41.262200,...,9.37,30.63,7.74,1028,205,3.731,1,147,1022.004500,1023.843239


In [13]:
neamap.to_csv("Data/neamap_longfin.csv")